In [ ]:
# 2025-12-16 run against production
from apis_core.apis_metainfo.models import Collection, Uri
from apis_core.apis_entities.models import Event, Place, Institution
from apis_core.apis_vocabularies.models import EventType, InstitutionEventRelation, PlaceEventRelation
from apis_core.apis_relations.models import InstitutionEvent, PlaceEvent
import glob
import json
from tqdm import tqdm
from datetime import datetime, date

In [ ]:
domain = "burgtheater"
base_url = "https://kulturerbe.burgtheater.at/event/"
mapping = {
    "Neueinstudierung": "Theaterneueinstudierung",
    "Premiere": "Theaterpremiere",
    "Erstaufführung": "Theatererstaufführung",
    "Neuinszenierung": "Theaterneuinszenierung",
    "Uraufführung": "Theateruraufführung",
    "Theateraufführung": "Theateraufführung"
}

In [ ]:
col, _ = Collection.objects.get_or_create(name="Burgtheater")

In [ ]:
files = glob.glob("/home/csae8092/repos/burgtherater-event-crawler/data/events/*.json")

In [ ]:
start = date(1850, 1, 1)
end = date(1938, 12, 31)
burgtheater = Place.objects.get(id=14)
burgtheater_inst = Institution.objects.get(id=36247)
bt_alt = Place.objects.get(id=185621)
veranstaltet = InstitutionEventRelation.objects.get(name="veranstaltet")
ort_event = PlaceEventRelation.objects.get(name="Veranstaltungsort von")
akademietheater = Place.objects.get(name="Akademietheater")


In [ ]:
items = []
for x in tqdm(files, total=len(files)):
    with open(x, "r", encoding="utf-8") as fp:
        data = json.load(fp)
        for y in data["productions"]:
            item = {}
            title = y["title"]
            date_str = y["date"].split("T")[0]
            parsed_date = datetime.strptime(date_str, "%Y-%m-%d").date()
            if start <= parsed_date <= end:
                date_formatted = f"{int(date_str.split('-')[2])}.{int(date_str.split('-')[1])}.{date_str.split('-')[0]}"
                uri = f"{base_url}{y['_id']}"
                kind = y["premiere"]
                if kind:
                    pass
                else:
                    kind = "Theateraufführung"
                pmb_kind = mapping[kind]
                event_type, _ = EventType.objects.get_or_create(name=pmb_kind)
                item["name"] = f"{kind} von {title}, {date_formatted}"
                item["start_date_written"] = date_str
                item["end_date_written"] = date_str
                item["kind"] = event_type
                entity, _ = Event.objects.get_or_create(**item)
                entity.collection.add(col)
                pmb_uri = Uri.objects.get_or_create(uri=uri, domain=domain, entity=entity)
                items.append(item)
                # link to institution
                InstitutionEvent.objects.get_or_create(
                    related_institution=burgtheater_inst,
                    related_event=entity,
                    start_date_written=date_str,
                    end_date_written=date_str,
                    relation_type=veranstaltet
                )
                # link to place
                if parsed_date > date(1888, 10, 12):
                    place = burgtheater
                else:
                    place = bt_alt
                if y["location"] == "Burgtheater":
                    place = place
                else:
                    place = akademietheater
                PlaceEvent.objects.get_or_create(
                    related_place=place,
                    related_event=entity,
                    start_date_written=date_str,
                    end_date_written=date_str,
                    relation_type=ort_event
                )